# Create a connection

The process of establishing a connection between two peers, referred to as the Inviter and Invitee, begins with the Inviter creating an out-of-band (oob) invitation. This invitation contains all the necessary information for the Invitee to connect with the Inviter. Once the invitation is created, it must be handed to the Invitee in some way, such as through email, messaging or QR code. Once the Invitee receives the invitation, they must accept it in order to proceed with the connection. Finally, with the invitation accepted, the connection is established, allowing the two peers to communicate and share data.

In [ ]:
import os
import datetime
from pprint import pprint
from dotenv import load_dotenv

from prism_agent_open_api_specification_client import Client
from prism_agent_open_api_specification_client.models import ConnectionCollection,Connection,ConnectionInvitation,CreateConnectionRequest,AcceptConnectionInvitationRequest
from prism_agent_open_api_specification_client.api.connections_management import get_connections,get_connection,create_connection,accept_connection_invitation
from prism_agent_open_api_specification_client.types import Response

### Utilitary functions

In [ ]:
def get_invitation_str(connection):
    parts = connection.invitation.invitation_url.split("=")
    return parts[1]

def print_connection(connection):
    print(f"connection_id: {connection.connection_id}")
    print(f"state:         {connection.state}")
    print(f"label:         {connection.label}")
    print(f"my_did:        {connection.my_did}")
    print(f"their_did:     {connection.their_did}")
    print(f"created_at:    {connection.created_at}")
    print(f"OOB Invitation: {get_invitation_str(connection)}")

### Client instances

We will create two separate clients, one for the inviter and one for the invitee, in order to establish a connection between the two. The roles of the inviter and invitee may overlap with the traditional holder, prover, issuer, and verifier relationships. In some cases, the inviter may also have the role of an issuer or verifier, while the invitee may have the role of a holder or prover. This flexibility allows for different scenarios and use cases to be supported within the same flow

note: remember to update the file variables.env with the URLs and API keys provided to you.


In [ ]:
load_dotenv("../BetaProgram/variables.env")
inviterApiKey = os.getenv('ISSUER_APIKEY')
inviterUrl = os.getenv('ISSUER_URL')

inviteeApiKey = os.getenv('HOLDER_APIKEY')
inviteeUrl = os.getenv('HOLDER_URL')

inviter_client = Client(base_url=inviterUrl, headers={"apiKey": inviterApiKey})
invitee_client = Client(base_url=inviteeUrl, headers={"apiKey": inviteeApiKey})

### Inviter - Create the invitation

An invitation is created when a connection is created. The only parameter required is a label to identify the connection with a human-readable format. 

In [ ]:
conn_request = CreateConnectionRequest()
conn_request.label = f'Connect {datetime.date.today().strftime("%Y-%m-%d %H:%M:%S")}'
inviter_connection: Response[Connection] =  create_connection.sync(client=inviter_client,json_body=conn_request)

invitation = get_invitation_str(inviter_connection)

Details of the connection on the inviter side are presented below. The connection contains the invitation in the invitation_url field.

In [ ]:
print_connection(inviter_connection)

### Invitee - Accept the invitation 

When the inviter creates the invitation, there is no connection record on the invitee side, which is why the invitation is shared out of band. Here we conveniently use a variable to pass the initation to the invitee

In [ ]:
accept_conn_request = AcceptConnectionInvitationRequest(invitation)
invitee_connection: Response[ConnectionInvitation] =  accept_connection_invitation.sync(client=invitee_client,json_body=accept_conn_request)


Details of the connection on the invitee side are presented below.

In [ ]:
print_connection(invitee_connection)

### Inviter - Check connection

The states of the connection on the inviter side are as described below:

InvitationGenerated --> ConnectionRequestReceived --> ConnectionResponsePending --> **ConnectionResponseSent**

The PRISM Agent does the process to establish the connection automatically, so it may not be possible to track all the protocol steps.

In [ ]:
inviter_connection: Response[Connection] =  get_connection.sync(client=inviter_client,connection_id=inviter_connection.connection_id)

In [ ]:
print_connection(inviter_connection)

### Invitee - Check connection

The states of the connection on the inviter side are as described below:

InvitationReceived  --> ConnectionRequestPending  --> ConnectionRequestSent     --> **ConnectionResponseReceived**

The PRISM Agent does the process to establish the connection automatically, so it may not be possible to track all the protocol steps.

In [ ]:
invitee_connection: Response[Connection] =  get_connection.sync(client=invitee_client,connection_id=invitee_connection.connection_id)

In [ ]:
print_connection(invitee_connection)

### List all connections

The following code retrieves the lists of connections.

In [ ]:
inviter_connections: Response[ConnectionCollection] = get_connections.sync(client=inviter_client)
invitee_connections: Response[ConnectionCollection] = get_connections.sync(client=invitee_client)
print("Inviter connections")
print("-------------------\n")
pprint("{}".format(inviter_connections, invitee_connections))
print("\nInvitee connections")
print("-------------------\n")
pprint("{}".format(invitee_connections, invitee_connections))